In [1]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
from utils import iter_df_read, get_array_diff
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("seaborn")
sns.set(font_scale=1.5)

# Modificar por ruta local de almacenamiento de datos
BASE = "/media/giani/Gianicosas/Magister/Proyecto/"

# Atributos primera iteración
- **Notas de colegio**: Egresados Colegio, `PROM_NOTAS_ALU`
- **Región del colegio**: Egresados Colegio, `COD_REG_RBD`
- **Modalidad del colegio (científico humanista / técnico)**: Egresados Colegio, `COD_ENSE`
- **Tipo de establecimiento (particular, subvencionado, municipal)**: Establecimientos, 
- **Género**: Matriculados, `gen_alu`.
- **Edad**: Matriculados, `fec_nac_alu`.
- **Área del conocimiento de la carrera**: Matriculados, `area_conocimiento`.
- **Región de la institución de educación superior**: Matriculados, `region_sede`.

# Lectura datos

## Desertores

In [2]:
desertores_1 = pd.read_csv(f"{BASE}/tmp_data/desertores_1.csv.gz")
desertores_1.head()

,cat_periodo,codigo_unico,mrun,desertor_1
0,2015,I374S1C9J1V1,5.0,0
1,2015,I498S6C132J2V1,37.0,0
2,2015,I111S14C309J2V1,118.0,0
3,2015,I260S44C120J2V1,184.0,0
4,2015,I106S3C64J4V1,253.0,1


## Matrículas

In [3]:
cols_mat = [
    'cat_periodo', 'codigo_unico', 'mrun', 'tipo_inst_1', 
    'gen_alu', 'fec_nac_alu', 'area_conocimiento', 'region_sede'
]

instituciones = ['Institutos Profesionales', 'Centros de Formación Técnica']

matriculados = pd.concat([
    iter_df_read(f, "tipo_inst_1", instituciones, sep=";", usecols=cols_mat)
    for f in tqdm(
        glob(f"{BASE}/raw_data/post_2015/20220719_Matrícula_Ed_Superior_*.csv"),
        total=8
    )
]).dropna(subset="mrun")

print(f"Cantidad de registros: {matriculados.shape[0]}\n")
print(f"Cantidad de nulos: \n{matriculados.isna().sum()}")
matriculados.head()

  0%|          | 0/8 [00:00<?, ?it/s]

Cantidad de registros: 4131609

Cantidad de nulos: 
cat_periodo          0
codigo_unico         0
mrun                 0
gen_alu              0
fec_nac_alu          0
tipo_inst_1          0
region_sede          0
area_conocimiento    0
dtype: int64


,cat_periodo,codigo_unico,mrun,gen_alu,fec_nac_alu,tipo_inst_1,region_sede,area_conocimiento
0,2015,I374S1C9J1V1,5.0,2,199105,Centros de Formación Técnica,Metropolitana,Salud
1,2015,I498S6C132J2V1,37.0,1,197808,Centros de Formación Técnica,Biobío,Tecnología
5,2015,I111S12C104J1V1,107.0,1,199409,Institutos Profesionales,Metropolitana,Tecnología
7,2015,I111S14C309J2V1,118.0,1,199102,Institutos Profesionales,Metropolitana,Administración y Comercio
9,2015,I260S44C120J2V1,184.0,1,199403,Centros de Formación Técnica,Magallanes,Administración y Comercio


## Colegios

Cruzar con https://datosabiertos.mineduc.cl/directorio-de-establecimientos-educacionales/ para sacar si es municipal, sub, etc

In [ ]:
cols_colegio = [
    'MRUN', 'PROM_NOTAS_ALU', 'COD_REG_RBD', 'RBD', 'COD_ENSE'
]

egreso_colegio = pd.concat([
    iter_df_read(f, "MRUN", desertores_1["mrun"].unique(), sep=";", usecols=cols_colegio)
    for f in tqdm(
        glob(f"{BASE}raw_data/egresados_colegios/20220325_Notas_y_Egresados_Enseñanza_Media_*.csv"),
        total=20
    )
]).dropna(subset="MRUN").drop_duplicates().reset_index(drop=True)

print(f"Cantidad de registros: {egreso_colegio.shape[0]}\n")
print(f"Cantidad de nulos: \n{egreso_colegio.isna().sum()}")
egreso_colegio.head()